In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import sys, platform, os
import copy as cp
import numpy as np 
import matplotlib.pyplot as plt
from ksz_model import KSZ_power
from reio_mod import xe_asym, xe_asym2, xe2tau, xe_tanh
from scipy.interpolate import interp1d
from matplotlib import cm, colors, rcParams

ModuleNotFoundError: No module named 'camb'

In [ ]:
camb_path = '/Users/lisaleemcb/CAMB' # os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.append(camb_path)

print(sys.path)
import camb
from camb import model, initialpower
print('Using CAMB %s installed at %s'%(camb.__version__,os.path.dirname(camb.__file__)))

In [ ]:
plt.style.use('seaborn-v0_8-colorblind')
plt.rc('text', usetex=True)
plt.rc('font', family='serif')

palette = plt.rcParams['axes.prop_cycle'].by_key()['color']
ls_list = ['-', '--', ':', '-.']

In [ ]:
rcParams.update({'font.size': 16})
rcParams['axes.linewidth'] = 1.8
plt.ion()

In [ ]:
from astropy.cosmology import Planck18

z = np.linspace(0, 20, 500)
ells = np.linspace(100, 10000, 20)

z_recomb = 1100.0
z3 = np.linspace(0, z_recomb, 10000)

### Comparison of Adélie's module and CAMB's code

In [ ]:
zre_H = 7.
dz_H = .5
zend_H = zre_H - dz_H

zre_HeI = 6.0
zre_HeII = 3.5
dz_HeI = 0.5
dz_HeII = 0.5
zre_HeI_start = 10.0
zre_HeII_start = 10.0

In [ ]:
z_CAMB = np.loadtxt('z_CAMB.txt')
tgh_CAMB = np.loadtxt('tgh_CAMB.txt')

In [ ]:
#Set up a new set of parameters for CAMB. This will be our fiducial model
pars = camb.CAMBparams()
#This function sets up CosmoMC-like settings, with one massive neutrino and helium set using BBN consistency
pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0)
pars.InitPower.set_params(As=2e-9, ns=0.965, r=0)
pars.set_for_lmax(2500, lens_potential_accuracy=0);

In [ ]:
pars.Reion

In [ ]:
pars.Reion.redshift = zre_H
pars.Reion.delta_redshift = delta_H
pars.Reion.max_redshift = 50.0

pars.Reion.heliumI_redshift = heliumI_redshift
pars.Reion.heliumI_delta_redshift = heliumI_delta_redshift

pars.Reion.heliumII_redshift = heliumII_redshift
pars.Reion.heliumII_delta_redshift = heliumII_delta_redshift

pars.Reion.include_heliumI_fullreion = True
pars.Reion.include_heliumII_fullreion = True

pars.Reion

In [ ]:
np.arange(.1,.5,.05), np.arange(.005,.25, .05)

In [ ]:
results = camb.get_results(pars)
x_e = results.get_background_redshift_evolution(z_CAMB, vars=['x_e'])

In [ ]:
xe_Adelie = xe_tanh(z_CAMB, ze=zre_H, deltaz=delta_H, helium1=True, helium2=True,
                        helium1_redshift=heliumI_redshift, helium1_deltaredshift=heliumI_delta_redshift,
                        helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,)

In [ ]:
zre_H, heliumI_redshift

In [ ]:
plt.plot(z, xe_tanh(z, ze=zre_H, deltaz=delta_H, helium1=True, helium2=True,
                        helium1_redshift=heliumI_redshift, helium1_deltaredshift=heliumI_delta_redshift,
                        helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,))

plt.plot(z, xe_tanh(z, ze=7, deltaz=.5, helium1=True, helium2=True,
                        helium1_redshift=7, helium1_deltaredshift=heliumI_delta_redshift,
                        helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,))

plt.plot(z, xe_tanh(z, ze=7, deltaz=.5, helium1=True, helium2=True,
                        helium1_redshift=3.5, helium1_deltaredshift=heliumI_delta_redshift,
                        helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,))

In [ ]:
fig, ax = plt.subplots(1,2, figsize=(12,5))

ax[0].plot(z_CAMB, xe_Adelie, label='Adelie')
ax[0].plot(z_CAMB, x_e['x_e'], label='CAMB', ls=':', color=palette[2])
#ax[0].plot(z_CAMB, tgh_CAMB, label='CAMB tgh', ls=':', color=palette[2])

ax[1].plot(z_CAMB, (xe_Adelie / x_e['x_e']), label='difference (Adelie / CAMB)')
ax[1].axvline(zre_H)
ax[1].axvline(heliumI_redshift)
ax[1].axvline(heliumII_redshift)

#ax[1].set_ylim(-2.164667223247818e-09, 1e-8)

ax[0].legend(fontsize=12)
ax[1].legend(fontsize=12)

#ax[1].set_ylim(-5,1)

fig.tight_layout()

### Reionization histories consistent with the Planck '18 measurement of, $\tau$, the optical depth

In [ ]:
zend_list_H = np.linspace(6, 8, 20)
zend_list_He = np.linspace(3.5, 8, 22)

zends = np.zeros((zend_list_H.size, zend_list_He.size, 2))

taus_tanh = np.zeros((zend_list_H.size, zend_list_He.size))
taus_asym = np.zeros((zend_list_H.size, zend_list_He.size))

for i in range(zend_list_H.size):
    for j in range(zend_list_He.size):
        zends[i,j,:] = zend_list_H[i], zend_list_He[j]
        if zend_list_H[i] > zend_list_He[j]:
            #print(zends[i,j])
            taus_tanh[i,j] = xe2tau(z, xe_tanh(z,ze=zend_list_H[i], helium1_redshift=zend_list_He[j]))[0]
            taus_asym[i,j] = xe2tau(z, xe_asym2(z, H_zend=zend_list_H[i], He_zend=zend_list_He[j],
                                             helium1=True, helium2=True))[0]

            x_e = xe_tanh(z, ze=zend_list_H[i], deltaz=.5, helium1=True, helium2=True,
                        helium1_redshift=zend_list_He[j], helium1_deltaredshift=.5,
                        helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,)
            plt.plot(z, x_e)

In [ ]:
for j in range(zend_list_He.size):
    zends[i,j,:] = zend_list_H[i], zend_list_He[j]
    if zend_list_H[i] > zend_list_He[j]:
        #print(zends[i,j])
        taus_tanh[i,j] = xe2tau(z, xe_tanh(z,ze=zend_list_H[i], helium1_redshift=zend_list_He[j]))[0]
        taus_asym[i,j] = xe2tau(z, xe_asym2(z, H_zend=zend_list_H[i], He_zend=zend_list_He[j],
                                         helium1=True, helium2=True))[0]

        x_e = xe_tanh(z, ze=zend_list_H[i], deltaz=.5, helium1=True, helium2=True,
                    helium1_redshift=zend_list_He[j], helium1_deltaredshift=.5,
                    helium2_redshift=heliumII_redshift, helium2_deltaredshift=heliumII_delta_redshift,)
        plt.plot(z, x_e)

In [ ]:
tau_Planck = 0.0540
sigma_Planck = 0.0074
sigma_Litebird = 0.002

#tau_asym_mask = (taus_asym > 0.0540 - 0.0074) & (taus_asym < 0.0540 + 0.0074)
tau_tanh_mask_Planck = (taus_tanh > tau_Planck - sigma_Planck) & (taus_tanh < tau_Planck + sigma_Planck)
tau_tanh_mask_Litebird = (taus_tanh > tau_Planck - sigma_Litebird) & (taus_tanh < tau_Planck + sigma_Litebird)

# referencing https://arxiv.org/pdf/2202.02773.pdf Litebird Fig. 47

In [ ]:
taus_tanh[tau_tanh_mask_Litebird]

In [ ]:
zend_list_H[np.where(tau_tanh_mask_Litebird == True)[0]]

In [ ]:
#from mpltools import color

#n_lines = 16
#color.cycle_cmap(n_lines, ax=ax[0])

colors = plt.cm.Spectral(np.linspace(0,1,25))

fig, ax = plt.subplots(1, 2, sharey=True, figsize=(12,5))

for i in range(zend_list_He.size):
    z_H = []
    taus = []
    for j in range(zend_list_H.size):
        #print(zend_list_H[i], zend_list_He[j])
        if zend_list_H[j] > zend_list_He[i]:
            z_H.append(zend_list_H[j])
            taus.append(taus_tanh[j,i])
    
    ax[0].plot(z_H, taus, label=r'$z_{\mathrm{HeI}}=$' + str(zend_list_He[i])[:3], color=colors[i])

    ax[0].set_xlabel('redshift of HI reionization')
    ax[0].set_ylabel('optical depth')
    
for i in range(zend_list_H.size):
    z_He = []
    taus = []
    for j in range(zend_list_He.size):
        #print(zend_list_H[i], zend_list_He[j])
        if zend_list_H[i] > zend_list_He[j]:
            z_He.append(zend_list_He[j])
            taus.append(taus_tanh[i,j])
    
    ax[1].plot(z_He, taus, label=r'$z_{\mathrm{HI}}=$' + str(zend_list_H[i])[:3], color=colors[i])

    ax[1].set_xlabel('redshift of HeI reionization')
  #  ax[1].set_ylabel('optical depth')
    
ax[0].axhline(tau_Planck, color='black', alpha=.5)
ax[1].axhline(tau_Planck, color='black', alpha=.5)

ax[0].fill_between(zend_list_H, np.ones_like(zend_list_H) * tau_Planck,
                       np.ones_like(zend_list_H) * (tau_Planck + sigma_Planck), color='gray', alpha=.2)
ax[0].fill_between(zend_list_H, np.ones_like(zend_list_H) * tau_Planck,
                       np.ones_like(zend_list_H) * (tau_Planck - sigma_Planck), color='gray', alpha=.2)

ax[1].fill_between(zend_list_He, np.ones_like(zend_list_He) * tau_Planck,
                       np.ones_like(zend_list_He) * (tau_Planck + sigma_Planck), color='gray', alpha=.2)
ax[1].fill_between(zend_list_He, np.ones_like(zend_list_He) * tau_Planck,
                       np.ones_like(zend_list_He) * (tau_Planck - sigma_Planck), color='gray', alpha=.2)

ax[0].set_xlim(zend_list_H[0], zend_list_H[-1])
ax[1].set_xlim(zend_list_He[0], zend_list_He[-1])

ax[0].legend(fontsize=12)
ax[1].legend(fontsize=12)

fig.tight_layout()

In [ ]:
import matplotlib.colors 

fig, ax = plt.subplots(1,2,figsize=(10,15))

im = ax[0].imshow(taus_tanh, extent=[zend_list_He[0], zend_list_He[-1],
                                             zend_list_H[0], zend_list_H[-1]],
               norm=matplotlib.colors.CenteredNorm(tau_Planck))

im2 = ax[1].imshow(taus_asym, extent=[zend_list_He[0], zend_list_He[-1],
                                             zend_list_H[0], zend_list_H[-1]],
                norm=matplotlib.colors.CenteredNorm(tau_Planck))

# im3 = ax[2].imshow(taus_asym / taus_tanh, extent=[zend_list_He[0], zend_list_He[-1],
#                                              zend_list_H[0], zend_list_H[-1]],
#                cmap=cmap, vmin=1e-5)

#labels_H = [str(z) for z in zend_list_H]

# Show all ticks and label them with the respective list entries
#ax.set_xticks(np.arange(len(zend_list_He)))#, labels=str(zend_list_He))
#ax.set_yticks(np.arange(len(zend_list_H)), labels=labels_H)

# Rotate the tick labels and set their alignment.
# plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#          rotation_mode="anchor")


# # Loop over data dimensions and create text annotations.
# for i in range(len(zend_list_H)):
#         for j in range(len(zend_list_He)):
#             text = ax.text(j, i, zends[i, j],
#                             ha="center", va="center", color="w", fontsize=12)

ax[0].set_xlabel('redshift of He')
ax[1].set_xlabel('redshift of He')

ax[0].set_ylabel('redshift of H')

#fig.colorbar(im)

In [ ]:
import matplotlib as mpl

fig, ax = plt.subplots(1,2,figsize=(10,5))

cmap = mpl.cm.get_cmap("viridis").copy()
cmap.set_under(color='black')   
cmap.set_bad(color='white')

taus_Planck_masked = np.ma.masked_where(((taus_tanh > tau_Planck + sigma_Planck) > 0) | ((taus_tanh < tau_Planck - sigma_Planck) > 0), taus_tanh)
taus_Litebird_masked = np.ma.masked_where(((taus_tanh > tau_Planck + sigma_Litebird) > 0) | ((taus_tanh < tau_Planck - sigma_Litebird) > 0), taus_tanh)

im = ax[0].imshow(taus_Planck_masked, extent=[zend_list_He[0], zend_list_He[-1],
                                             zend_list_H[0], zend_list_H[-1]],
               cmap=cmap, vmin=1e-5, aspect='auto')

im2 = ax[1].imshow(taus_Litebird_masked, extent=[zend_list_He[0], zend_list_He[-1],
                                             zend_list_H[0], zend_list_H[-1]],
               cmap=cmap, vmin=1e-5, aspect='auto')

#labels_H = [str(z) for z in zend_list_H]

# Show all ticks and label them with the respective list entries
#ax.set_xticks(np.arange(len(zend_list_He)))#, labels=str(zend_list_He))
#ax.set_yticks(np.arange(len(zend_list_H)), labels=labels_H)

# Rotate the tick labels and set their alignment.
# plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
#          rotation_mode="anchor")


# # Loop over data dimensions and create text annotations.
for i in range(len(zend_list_H)):
        for j in range(len(zend_list_He)):
            text = ax[0].text(j, i, zends[i, j],
                            ha="center", va="center", color="w", fontsize=12)

ax[0].set_xlabel('redshift of He')
ax[1].set_xlabel('redshift of He')

ax[0].set_ylabel('redshift of H')

ax[0].set_title('Planck constraints')
ax[1].set_title('Litebird constraints')

#fig.colorbar(im)

In [ ]:
zends.shape, zend_list_He.shape, zend_list_H.shape

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(taus_Planck_masked)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(zend_list_He)),
                      labels=[str(zend_list_He[i])[:4] for i in range(len(zend_list_He))])
ax.set_yticks(np.arange(len(zend_list_H)), 
                      labels=[str(zend_list_H[i])[:4] for i in range(len(zend_list_H))])

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(zend_list_He)):
    for j in range(len(zend_list_H)):
        text = ax.text(j, i, taus_Planck_masked[j,i],
                       ha="center", va="center", color='pink', fontsize=10)

ax.set_title("Harvest of local farmers (in tons/year)")
fig.tight_layout()

In [ ]:
str(taus_tanh[10,1])[1:6], tau_Planck + sigma_Litebird

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(taus_Planck_masked)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(zend_list_He)),
                      labels=[str(zend_list_He[i])[:4] for i in range(len(zend_list_He))])
ax.set_yticks(np.arange(len(zend_list_H)), 
                      labels=[str(zend_list_H[i])[:4] for i in range(len(zend_list_H))])

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(zend_list_H)):
    for j in range(len(zend_list_He)):
        text = ax.text(j, i, str(taus_tanh[i, j])[1:6],
                       ha="center", va="center", color="k", fontsize=10)

ax.set_title("Planck allowed optical depths")
fig.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
im = ax.imshow(taus_Litebird_masked)

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(zend_list_He)),
                      labels=[str(zend_list_He[i])[:4] for i in range(len(zend_list_He))])
ax.set_yticks(np.arange(len(zend_list_H)), 
                      labels=[str(zend_list_H[i])[:4] for i in range(len(zend_list_H))])

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(zend_list_H)):
    for j in range(len(zend_list_He)):
        text = ax.text(j, i, str(taus_tanh[i, j])[1:6],
                       ha="center", va="center", color="magenta", fontsize=11)

ax.set_title("Litebird allowed optical depths")
fig.tight_layout()
plt.show()

### Impact of reionization on CMB spectra

In [ ]:
def gen_CMB_from_reion(z, H_redshift, HeI_redshift, heliumI_redshiftstart=10.0):
    # Set up a new set of parameters for CAMB. This will be our fiducial model
    pars = camb.CAMBparams()
    # This function sets up CosmoMC-like settings, with one massive neutrino 
    # and helium set using BBN consistency
    pars.set_cosmology(H0=67.5, ombh2=0.022, omch2=0.122, mnu=0.06, omk=0) #, tau=0.0544)
    pars.InitPower.set_params(As=2e-9, ns=0.965, r=0)
    pars.set_for_lmax(2500, lens_potential_accuracy=0)
    
   # print(pars.Reion.optical_depth)
    
    pars.Reion.use_optical_depth = False
    pars.Reion.redshift = H_redshift
    pars.Reion.heliumI_redshift = HeI_redshift
    pars.Reion.heliumI_delta_redshift  = .5
    pars.Reion.heliumI_redshiftstart  = heliumI_redshiftstart

    results = camb.get_results(pars)
       
    x_e = results.get_background_redshift_evolution(z, vars=['x_e'])
    powers = results.get_cmb_power_spectra(pars, CMB_unit='muK')
    
    totCL = powers['total']
    
    #print(pars.Reion.heliumI_redshift)
    return x_e['x_e'], totCL

def plot_CMB_from_reion(ax, totCL, fiducial, color, ls, label):
    ells = np.arange(totCL.shape[0])
    
    ax[0,0].plot(ells,totCL[:,0], color=color, ls=ls, label=label)
    ax[0,1].plot(ells,totCL[:,1], color=color, ls=ls, label=label)
    ax[0,2].plot(ells,totCL[:,2], color=color, ls=ls, label=label)
    ax[0,3].plot(ells,totCL[:,3], color=color, ls=ls, label=label)

    ax[1,0].plot(ells[2:], 1-totCL[2:,0]/fiducial[2:,0], color=color, ls=ls)
    ax[1,1].plot(ells[2:], 1-totCL[2:,1]/fiducial[2:,1], color=color, ls=ls)
    ax[1,2].plot(ells[2:], 1-totCL[2:,2]/fiducial[2:,2], color=color, ls=ls)
    ax[1,3].plot(ells[2:], 1-totCL[2:,3]/fiducial[2:,3], color=color, ls=ls)

# ax[0,0].plot(ells,totCL[:,0], color='k')
# ax[0,0].plot(ells,unlensedCL[:,0], color='r')
# ax[0,0].set_title('TT')
# ax[0,1].plot(ells[2:], 1-unlensedCL[2:,0]/totCL[2:,0]);
# ax[0,1].set_title(r'$\Delta TT$')
# ax[1,0].plot(ells,totCL[:,1], color='k')
# ax[1,0].plot(ells,unlensedCL[:,1], color='r')
# ax[1,0].set_title(r'$EE$')
# ax[1,1].plot(ells,totCL[:,3], color='k')
# ax[1,1].plot(ells,unlensedCL[:,3], color='r')
# ax[1,1].set_title(r'$TE$');
# for ax in ax.reshape(-1): ax.set_xlim([2,2500]);

 #   [ax[1,i].set_yscale('log') for i in range(ax[1,:].size)]

    ax[0,0].set_title('$TT$')
    ax[0,1].set_title('$EE$')
    ax[0,2].set_title('$BB$')
    ax[0,3].set_title('$TE$')

    ax[1,0].set_title('$\Delta TT$')
    ax[1,1].set_title('$\Delta EE$')
    ax[1,2].set_title('$\Delta BB$')
    ax[1,3].set_title('$\Delta TE$')

In [ ]:
zends[tau_tanh_mask_Planck].size

In [ ]:
Cls_list = np.zeros((len(zends[tau_tanh_mask_Planck]), 2551, 4))
zH_old = 0
p_i = -1

print('tau min is', tau_Planck - sigma_Planck, 'and tau max is', tau_Planck + sigma_Planck)
for i in range(len(zends[tau_tanh_mask_Planck])):
    zH, zHe = zends[tau_tanh_mask_Planck][i]
    if zH == zH_old:
        p_i += 1
        if p_i >= len(palette):
            p_i = 0
    zH_old = cp.deepcopy(zH)
#     plt.plot(z[50:300], xe_tanh(z,ze=zH, deltaz=0.5, helium1=True, helium1_redshift=zHe,
#                         helium1_deltaredshift=.5,
#                         helium2=True)[50:300], color=palette[0], alpha=.25)
    
#     plt.plot(z[50:300], xe_asym2(z, H_zend=zH, He_zend=zHe, helium1=True, helium2=True)[50:300],
#                                color=palette[1], alpha=.25)
    
    x_e, Cls_list[i] =  gen_CMB_from_reion(z, zH, zHe)
    # print(xe2tau(z, x_e)[0])
    plt.plot(z[50:300], x_e[50:300], color=palette[p_i], alpha=.1)


In [ ]:
fig, ax = plt.subplots(2,4, figsize=(10,8))

fiducial = gen_CMB_from_reion(z, 7, 5.5)[1]
ells = np.arange(fiducial.shape[0])

ell_max = 1000

ax[0,0].set_xlim(0, ell_max)

ax[0,1].set_xlim(0, 50)
ax[0,1].set_ylim(-.05, .2)

ax[0,2].set_xlim(0, 200)
ax[0,2].set_ylim(-.005, .02)

ax[0,3].set_xlim(0, 20)
ax[0,3].set_ylim(-1, 5)

ax[1,0].set_xlim(0, ell_max)
ax[1,1].set_xlim(0, 50)
ax[1,2].set_xlim(0, 200)
ax[1,3].set_xlim(0, 20)

# [ax[0,i].set_xlim(ells[10], ells[-1]) for i in range(4)]
# [ax[1,i].set_xlim(ells[10], ells[-1]) for i in range(4)]

# ax[1,0].set_ylim(-.005, 0.02)
# ax[1,1].set_ylim(-.005, 0.02)
# ax[1,2].set_ylim(-.005, 0.020)
# ax[1,3].set_ylim(-.005, .020)

fig.tight_layout()

#NUM_COLORS = len(zends[tau_tanh_mask])

# #cm = plt.get_cmap('gist_rainbow')
# ax.set_prop_cycle(color=[cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)])
# for i in range(NUM_COLORS):
#     ax.plot(np.arange(10)*(i+1))


for i in range(len(zends[tau_tanh_mask_Planck])):
    zH, zHe = zends[tau_tanh_mask_Planck][i]
    plot_CMB_from_reion(ax, Cls_list[i], fiducial, palette[i%5], '-', 'None')
 


In [ ]:
for i, zs in enumerate(zre_He):
    for j, dz in enumerate(deltaz_HeI):
        x_e, Cls =  gen_CMB_from_reion(z, zs, dz)
        ion_histories[i,j,:] = x_e
        spectras[i,j,:,:] = Cls

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(12,8))

for i, zs in enumerate(z_HeI):
    for j, dz in enumerate(deltaz_HeI):
        if j==0:
            label = 'z=' + str(zs) 
        else:
            label = '' 

        plot_CMB_from_reion(ax, spectras[i,j], spectras[2,1], palette[i],
                            ls_list[j], label)

ell_max = 1000

ax[0,0].set_xlim(0, ell_max)

ax[0,1].set_xlim(0, 50)
ax[0,1].set_ylim(-.05, .2)

ax[0,2].set_xlim(0, 200)
ax[0,2].set_ylim(-.005, .02)

ax[0,3].set_xlim(0, 20)
ax[0,3].set_ylim(-1, 5)

ax[1,0].set_xlim(0, ell_max)
ax[1,1].set_xlim(0, 50)
ax[1,2].set_xlim(0, 200)
ax[1,3].set_xlim(0, 20)

#ax[1,0].set_yscale('log')


ax[0,3].legend(fontsize=13, bbox_to_anchor=(1,1.09))



From here, it's clear that the dominant contribution to the change in CMB spectra is due to the redshift of hydrogen reionization. But what impact does the helium reionization have? Let's fix our redshift of hydrogen reionization, and zoom in on the helium redshifts.

In [ ]:
fig, ax = plt.subplots(2,4, figsize=(12,8))

i = 1
for j, dz in enumerate(deltaz_HeI):
    plot_CMB_from_reion(ax, spectras[i,j], spectras[i,1], palette[i],
                        ls_list[j], 'dz=' + str(dz) )

ell_max = 1000

ax[0,0].set_xlim(0, ell_max)

ax[0,1].set_xlim(0, 50)
ax[0,1].set_ylim(-.05, .2)

ax[0,2].set_xlim(0, 200)
ax[0,2].set_ylim(-.005, .02)

ax[0,3].set_xlim(0, 50)
ax[0,3].set_ylim(-1, 5)

ax[1,0].set_xlim(0, ell_max)

ax[1,1].set_xlim(0, 50)
ax[1,1].set_ylim(-.0004, .0004)

ax[1,2].set_xlim(0, 200)

ax[1,3].set_xlim(0, 50)
ax[1,3].set_ylim(-.0004, .0004)

#ax[1,0].set_yscale('log')


ax[0,3].legend(fontsize=13, bbox_to_anchor=(1,1.09))

fig.tight_layout()

### Impact of reionization on ksZ spectra

In [ ]:
Cls_shape = (2551,4)

In [ ]:
# zends[tau_asym_mask].shape, zends[tau_tanh_mask]

In [ ]:
import time
# H reion parameters
zre_H = 7.
zend_H = 5.8
xe_H = xe_asym(z, zend_H, zre_H,
               helium1=True,
               helium2=True)

# simultaneous H and He reion
ksz_sim = KSZ_power(dz = zre_H-zend_H, zre=zre_H,
                                    include_heliumI_fullreion=False,
                                    include_heliumII_fullreion=False)

In [ ]:
ksz_sim.init_reionisation_history()

In [ ]:
ksz_sim.run_camb()

In [ ]:
start = time.time()
Dls1 = ksz_sim.run_ksz(ells, Dells=True)
end = time.time()

In [ ]:
Dls_list[0][0][:,0]

In [ ]:
print('Time spent was', end-start, 'seconds')

In [ ]:
Dls.shape

In [ ]:
zends[tau_tanh_mask]

### ksZ simulations

In [ ]:
for i in range(len(zends[tau_tanh_mask])):
    plt.plot(ells, Dls_list[i,:,0])

In [ ]:
ells = np.linspace(100, 10000, 20)

Dls_list = np.zeros((15, ells.size, 2))
for i in range(len(zends[tau_tanh_mask])):
    print('Now on run', i, '...')
    zH, zHe = zends[tau_tanh_mask][i]
#     plt.plot(z[50:300], xe_tanh(z,ze=zH, deltaz=0.5, helium1=True, helium1_redshift=zHe,
#                         helium1_deltaredshift=.5,
#                         helium2=True)[50:300], color=palette[0], alpha=.25)
    
#     plt.plot(z[50:300], xe_asym2(z, H_zend=zH, He_zend=zHe, helium1=True, helium2=True)[50:300],
#                                color=palette[1], alpha=.25)
    ksz_sim = KSZ_power(dz=.5, zre=zH, heliumI_redshift=zHe, heliumI_delta_redshift=.5)
    ksz_sim.init_reionisation_history()
    ksz_sim.run_camb()
    Dls = ksz_sim.run_ksz(ells, Dells=True)
    
    Dls_list[i] = Dls


In [ ]:
for i in range(Dls_list.s)

In [ ]:
ksz_sim1 = KSZ_power(dz=.5, zre=7, heliumI_redshift=6.0, heliumI_delta_redshift=.5)
ksz_sim1.init_reionisation_history()
ksz_sim1.run_camb()
Dls1 = ksz_sim1.run_ksz(ells, Dells=True)

In [ ]:
plt.plot(ells, Dls1[:,0])
plt.plot(ells, Dls2[:,0])

In [ ]:
Dls1.shape, ells.shape

In [ ]:
ksz_sim2 = KSZ_power(dz=.5, zre=7, heliumI_redshift=5.5, heliumI_delta_redshift=.5)
ksz_sim2.init_reionisation_history()
ksz_sim2.run_camb()
Dls2 = ksz_sim2.run_ksz(ells, Dells=True)

In [ ]:
Add neutral hydrogen fraction